In [326]:
%run 'Load & Clean Liar'.ipynb

File Name                                             Modified             Size
README                                         2017-04-23 03:33:54         1670
test.tsv                                       2017-04-23 03:21:28       301118
train.tsv                                      2017-04-23 03:21:12      2408165
valid.tsv                                      2017-04-23 03:21:22       301556



Liar train:
Index(['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state',
       'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

Liar test:
Index(['id', 'label', 'statement', 'subject', 'speaker', 'job', 'state',
       'party', 'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c',
       'pants_on_fire_c', 'venue'],
      dtype='object')

Liar train:
(10240, 14)

Liar test:
(1267, 14)

Liar train:
id                 0
label              0
statement          0
subject            0
speaker       

In [327]:
liar_train_df['score'] = (liar_train_df.mostly_true_c * 0.9 + liar_train_df.half_true_c * 0.5 + \
                        liar_train_df.barely_true_c * 0.2 + liar_train_df.false_c * -1 + \
                        liar_train_df.pants_on_fire_c * -2).astype('int')

liar_test_df['score'] = (liar_test_df.mostly_true_c * 0.9 + liar_test_df.half_true_c * 0.5 + \
                        liar_test_df.barely_true_c * 0.2 + liar_test_df.false_c * -1 + \
                        liar_test_df.pants_on_fire_c * -2).astype('int')

In [328]:
liar_train_df = liar_train_df.drop('mostly_true_c', axis=1)
liar_train_df = liar_train_df.drop('half_true_c', axis=1)
liar_train_df = liar_train_df.drop('barely_true_c', axis=1)
liar_train_df = liar_train_df.drop('false_c', axis=1)
liar_train_df = liar_train_df.drop('pants_on_fire_c', axis=1)

In [329]:
liar_test_df = liar_test_df.drop('mostly_true_c', axis=1)
liar_test_df = liar_test_df.drop('half_true_c', axis=1)
liar_test_df = liar_test_df.drop('barely_true_c', axis=1)
liar_test_df = liar_test_df.drop('false_c', axis=1)
liar_test_df = liar_test_df.drop('pants_on_fire_c', axis=1)

In [330]:
liar_train_df.shape, liar_test_df.shape

((8920, 10), (1111, 10))

In [331]:
train_subj_columns = liar_train_df['subject'].str.split(',', expand=True).rename(columns=lambda x: f"subj_{x+1}")
test_subj_columns = liar_test_df['subject'].str.split(',', expand=True).rename(columns=lambda x: f"subj_{x+1}")

In [332]:
for column in train_subj_columns.columns:
    liar_train_df[column] = train_subj_columns[column]
liar_train_df = liar_train_df.drop('subject', axis=1)

for column in test_subj_columns.columns:
    liar_test_df[column] = test_subj_columns[column]
liar_test_df = liar_test_df.drop('subject', axis=1)

liar_train_df = liar_train_df.replace(r'^\s*$', np.nan, regex=True)
liar_train_df.fillna(value=np.nan, inplace=True)

liar_test_df = liar_test_df.replace(r'^\s*$', np.nan, regex=True)
liar_test_df.fillna(value=np.nan, inplace=True)

liar_train_df = liar_train_df.drop('id', axis=1)
liar_test_df = liar_test_df.drop('id', axis=1)

print(liar_train_df)

             label                                          statement  \
0            false  Says the Annies List political group supports ...   
1        half-true  When did the decline of coal start? It started...   
3            false  Health care reform legislation is likely to ma...   
4        half-true  The economic turnaround started at the end of ...   
5             true  The Chicago Bears have had more starting quart...   
...            ...                                                ...   
10235  mostly-true  There are a larger number of shark attacks in ...   
10236  mostly-true  Democrats have now become the party of the [At...   
10237    half-true  Says an alternative to Social Security that op...   
10238        false  On lifting the U.S. Cuban embargo and allowing...   
10239   pants-fire  The Department of Veterans Affairs has a manua...   

              speaker                                            job  \
0        dwayne-bohac                           Sta

In [333]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Keras models
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [337]:
from keras import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
import numpy as np

early_stopping_monitor = EarlyStopping(patience=2)

input_shape = (len(liar_train_df.columns),)

liar_NN_1 = Sequential()
liar_NN_1.add(Dense(liar_train_df.shape[1], activation='relu', input_shape=input_shape))
liar_NN_1.add(Dense(2, activation='softmax', input_shape=input_shape))

liar_NN_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
liar_NN_1 = liar_NN_1.fit(liar_train_df.drop('score', axis=1), liar_train_df.score, epochs=20, validation_split=0.4, callbacks=[early_stopping_monitor], verbose=False)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).